In [ ]:
!pip install -q requests beautifulsoup4 trafilatura

import requests
from bs4 import BeautifulSoup
import trafilatura
import time



UNIVERSITY_URLS = {
    "MIT-OCW": "https://ocw.mit.edu/courses/",
    "MIT-AI": "https://www.eecs.mit.edu/category/news/",
    "Stanford-AI": "https://ai.stanford.edu/",
    "Stanford-Online": "https://online.stanford.edu/courses",
    "Berkeley-EECS": "https://eecs.berkeley.edu/",
    "Berkeley-BAIR": "https://bair.berkeley.edu/blog/",
    "CMU-CS": "https://www.cs.cmu.edu/",
    "CMU-ML": "https://www.ml.cmu.edu/research/index.html"
}

PAGES_PER_URL = 300
OUTPUT_FILE = "usa_edu_corpus_cleaned.txt"
CRAWLED_URLS_FILE = "crawled_urls.txt"

BLOCK_EXTENSIONS = [".mp4", ".pdf", ".zip", ".jpg", ".png", ".ppt", ".doc", ".exe"]

#UTILITY FUNCTIONS

def is_valid_url(url):
    return not any(url.lower().endswith(ext) for ext in BLOCK_EXTENSIONS)

def get_all_links(page_url, base_domain):
    """Extract all internal links from a page."""
    try:
        response = requests.get(page_url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        links = set()
        for tag in soup.find_all("a", href=True):
            href = tag["href"]
            if href.startswith("/"):
                href = base_domain + href
            if base_domain in href and href.startswith("http") and is_valid_url(href):
                links.add(href.split("#")[0])
        return list(links)
    except Exception as e:
        print(f"[Error] Failed to get links from {page_url}: {e}")
        return []

def extract_clean_text(url):
    """Use trafilatura to extract clean main content from a page."""
    try:
        downloaded = trafilatura.fetch_url(url)
        if downloaded:
            return trafilatura.extract(downloaded)
        else:
            return None
    except Exception as e:
        print(f"[Error] Trafilatura failed for {url}: {e}")
        return None

#MAIN CRAWLER

def crawl_from_seed(name, seed_url, limit, out_f, log_f):
    seen = set()
    count = 0
    base_domain = seed_url.split("/")[0] + "//" + seed_url.split("/")[2]
    pages = [seed_url]

    print(f"\n====== Crawling {name} ======")
    for url in pages:
        if url in seen or count >= limit:
            continue
        seen.add(url)
        print(f"[{name} {count+1}] {url}")
        text = extract_clean_text(url)
        if text:
            paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) > 40]
            for p in paragraphs:
                out_f.write(p + "\n")
            log_f.write(url + "\n")
            count += 1
        time.sleep(1)
        new_links = get_all_links(url, base_domain)
        pages.extend([link for link in new_links if link not in seen])
        if count >= limit:
            break

    print(f"{name} done. {count} pages scraped.\n")

def crawl_all(university_urls, pages_per_url, output_file, crawled_log):
    with open(output_file, "w", encoding="utf-8") as out_f, open(crawled_log, "w", encoding="utf-8") as log_f:
        for name, url in university_urls.items():
            crawl_from_seed(name, url, pages_per_url, out_f, log_f)

#RUN SCRIPT

crawl_all(UNIVERSITY_URLS, PAGES_PER_URL, OUTPUT_FILE, CRAWLED_URLS_FILE)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 16.0 MB/s eta 0:00:00

====== Crawling MIT-OCW ======
[MIT-OCW 1] https://ocw.mit.edu/courses/
[MIT-OCW 2] https://ocw.mit.edu/about
[MIT-OCW 3] https://ocw.mit.edu/
[MIT-OCW 4] https://ocw.mit.edu/contact
[MIT-OCW 5] https://ocw.mit.edu/collections/mit-open-learning-library/
[MIT-OCW 6] https://ocw.mit.edu/search/?f=Online%20Textbook&s=-runs.best_start_date
[MIT-OCW 7] https://ocw.mit.edu/search/?f=Instructor%20Insights&s=-runs.best_start_date
[MIT-OCW 8] https://ocw.mit.edu/pages/privacy-and-terms-of-use/
[MIT-OCW 9] https://ocw.mit.edu/search/
[MIT-OCW 10] https://ocw.mit.edu/educator
[MIT-OCW 11] https://ocw.mit.edu/search/?t=Linear+Algebra
[MIT-OCW 12] https://ocw.mit.edu/search/?t=Lea

[MIT-OCW 84] https://ocw.mit.edu/search/?t=Mechanical+Engineering
[MIT-OCW 85] https://ocw.mit.edu/collections/ocw-scholar/
[MIT-OCW 86] https://ocw.mit.edu/search/?q=Prof.+Yufei+Zhao
[MIT-OCW 87] https://ocw.mit.edu/courses/2-79j-biomaterials-tissue-interactions-fall-2022/
[MIT-OCW 88] https://ocw.mit.edu/stories/ana-tri%C5%A1ovi%C4%87/
[MIT-OCW 89] https://ocw.mit.edu/stories/
[MIT-OCW 90] https://ocw.mit.edu/courses/2-785j-cell-matrix-mechanics-fall-2016/
[MIT-OCW 91] https://ocw.mit.edu/search/?q=Joe+Diaz
[MIT-OCW 92] https://ocw.mit.edu/search/?l=Graduate
[MIT-OCW 93] https://ocw.mit.edu/search/?t=Discrete+Mathematics
[MIT-OCW 94] https://ocw.mit.edu/courses/6-s087-foundation-models-and-generative-ai-january-iap-2024/
[MIT-OCW 95] https://ocw.mit.edu/search/?t=Microeconomics
[MIT-OCW 96] https://ocw.mit.edu/search/?q=Prof.+Steven+G.+Johnson
[MIT-OCW 97] https://ocw.mit.edu/search/?t=American+Politics
[MIT-OCW 98] https://ocw.mit.edu/courses/17-251-congress-and-the-american-politic

ERROR:trafilatura.downloads:not a 200 response: 403 for URL https://ocw.mit.edu/help/


[MIT-OCW 133] https://ocw.mit.edu/help/
[MIT-OCW 133] https://ocw.mit.edu/search/?q=Elizabeth+Huttner-Loan
[MIT-OCW 134] https://ocw.mit.edu/search/?q=Prof.+Justin+Reich
[MIT-OCW 135] https://ocw.mit.edu/search/?t=Finance
[MIT-OCW 136] https://ocw.mit.edu/courses/res-15-005-healthcare-finance-15-482x-spring-2019/
[MIT-OCW 137] https://ocw.mit.edu/search/?q=Prof.+Silvija+Gradecak
[MIT-OCW 138] https://ocw.mit.edu/search/?q=Prof.+Peter+Shor
[MIT-OCW 139] https://ocw.mit.edu/courses/8-370x-quantum-information-science-i-spring-2018/
[MIT-OCW 140] https://ocw.mit.edu/search/?t=Health+Care+Management
[MIT-OCW 141] https://ocw.mit.edu/search/?t=Materials+Science+and+Engineering
[MIT-OCW 142] https://ocw.mit.edu/search/?t=Algorithms+and+Data+Structures
[MIT-OCW 143] https://ocw.mit.edu/search/?t=Educational+Technology
[MIT-OCW 144] https://ocw.mit.edu/search/?q=Alyssa+Napier
[MIT-OCW 145] https://ocw.mit.edu/courses/res-cms-155-design-thinking-for-leading-and-learning-spring-2019/
[MIT-OCW 146

ERROR:trafilatura.downloads:not a 200 response: 404 for URL https://ai.stanford.edu/portfolio-view/dharmendra-modha/


...

✅ CMU-CS done. 236 pages scraped.


====== Crawling CMU-ML ======
[CMU-ML 1] https://www.ml.cmu.edu/research/index.html
[CMU-ML 2] https://www.ml.cmu.edu/academics/minor-senior-projects.html
[CMU-ML 3] https://www.ml.cmu.edu//www.cmu.edu/legal/
[CMU-ML 4] https://www.ml.cmu.edu/research/data-analysis-projects.html
[CMU-ML 5] https://www.ml.cmu.edu/people/core-faculty.html
[CMU-ML 6] https://www.ml.cmu.edu/research/phd-dissertations.html
[CMU-ML 7] https://www.ml.cmu.edu/research/technical-reports.html
[CMU-ML 8] https://www.ml.cmu.edu//www.cmu.edu/
[CMU-ML 9] https://www.ml.cmu.edu/index
[CMU-ML 10] https://www.ml.cmu.edu/Faculty_Hiring.html
[CMU-ML 11] https://www.ml.cmu.edu
[CMU-ML 12] https://www.ml.cmu.edu/current-students/student-orientation/


ERROR:trafilatura.utils:parsed tree length: 1, wrong data type or not valid HTML


[CMU-ML 13] https://www.ml.cmu.edu/cmsint/mldcmu.rss


ERROR:trafilatura.core:empty HTML tree: None


[CMU-ML 13] https://www.ml.cmu.edu/directions
[CMU-ML 14] https://www.ml.cmu.edu/about/contact-us.html
[CMU-ML 15] https://www.ml.cmu.edu/resources/
[CMU-ML 16] https://www.ml.cmu.edu/current-students/
[CMU-ML 17] https://www.ml.cmu.edu/current-students/frequently-asked-questions.html
✅ CMU-ML done. 17 pages scraped.

